# 데이터 및 라이브러리 셋팅

gdown or 캐글 데이터 다운 

In [ ]:
pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

예제 데이터(X-ray) - 변환을 원하는 이미지 다운로드하기

In [ ]:
!mkdir yolo_labels
!kaggle datasets download -d awsaf49/vinbigdata-yolo-labels-dataset
!kaggle datasets download -d awsaf49/vinbigdata-512-image-dataset

!unzip -d /content/yolo_labels /content/vinbigdata-yolo-labels-dataset.zip
!unzip /content/vinbigdata-512-image-dataset.zip

!rm /content/vinbigdata-512-image-dataset.zip
!rm /content/vinbigdata-yolo-labels-dataset.zip

라이브러리 불러오기

In [ ]:
!pip install albumentations==0.4.6
import albumentations 
from albumentations.pytorch import ToTensorV2

In [6]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import time
import json
import os
import glob
from PIL import Image
from tqdm import tqdm

import imgaug.augmenters as iaa
from sklearn.model_selection import GroupKFold

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

from torch.utils.data import DataLoader, Dataset

from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow


이미지 정보 들어있는 데이터프레임 핸들링

# 이미지 증강



1. Rotation 90도

2. Flip(horizontal)

3. Zoomin 10%

4. CLAHE(Contrast Limited Adaptive Histogram Equalization)

5. EqualizeHist

6. Cutmix

++ Mosaic






증강 이미지 생성

In [ ]:
# 나중에 함수로 만들것.
# 이미지 폴더 경로 넣기
# input_path = "Input your image path/*.image format(ex:jpg, png)"
# image_list = [f for f in glob.glob(input_path)]

In [ ]:
path = '/content/aug_image'
!mkdir /content/aug_image

## 1. Rotation 90도

In [ ]:
def rotation_image(path, image_list):
    for path in image_list:
        img = cv2.imread(path)
        (h, w) = img.shape[:2]
        (cX, cY) = (w/2, h/2)
        M = cv2.getRotationMatrix2D((cX,cY), 90, 1.0)
        rotated_img = cv2.warpAffine(img, M, (w,h))
        img_name = path.split('/')[-1].split('.')[0]
        cv2.imwrite(f'{path}/{img_name}'+'_rotated.png', rotated_img)


## 2. Flip(horizontal)

In [ ]:
def flip_image(path, image_list):
    for path in image_list:
        img = cv2.imread(path)
        flipped_img = cv2.flip(img, 1) 
        img_name = path.split('/')[-1].split('.')[0]
        cv2.imwrite(f'{path}/{img_name}'+'_flip.png', flipped_img)

## 3. Zoomin 10%

In [ ]:
def zoom_image(path, image_list):
    for path in image_list:
        img = cv2.imread(path)
        augment_img_zoom = iaa.Affine(scale=(1.1))
        zoom_image = augment_img_zoom.augment_image(img)
        img_name = path.split('/')[-1].split('.')[0]
        cv2.imwrite(f'{path}/{img_name}'+'_zoom.png', zoom_image)

# zoom_image(image_list)

## 4. CLAHE(Contrast Limited Adaptive Histogram Equalization)

In [ ]:
def clahe_image(path, image_list):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    for path in image_list:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        clahe_img = clahe.apply(img)
        img_name = path.split('/')[-1].split('.')[0]
        cv2.imwrite(f'{path}/{img_name}'+'_clahe.png', clahe_img)

## 5. EqualizeHist

In [ ]:
def equ_image(path, image_list):
    for path in image_list:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        equ_img = cv2.equalizeHist(img)
        img_name = path.split('/')[-1].split('.')[0]
        cv2.imwrite(f'{path}/{img_name}'+'_equ.png', equ_img)

## 6. Cutmix

In [ ]:
# 하나의 함수로 만들어보기.

def rand_bbox(size, lamb):
    W = size[0]
    H = size[1]
    cut_rat = np.sqrt(1. - lamb)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2


# REAL
# 우린 그럼 n_images를 4가아닌 총 images개수로 바꿔줘야겠지!
# Create batch of 4394 images
from tqdm import tqdm
def make_img_batch_labels(image_list):
    image_batch = []
    image_batch_labels = []
    n_images = len(image_list)

    for i in tqdm(range(n_images)):
        image = cv2.cvtColor(cv2.imread(image_list[i]), cv2.COLOR_BGR2RGB)
        image_batch.append(image)
    
        label_temp = list(np.floor(np.random.rand(1)*2.99).astype(int))[0]
        if label_temp == 0:
            label = [1,0,0]
        elif label_temp == 1:
            label = [0,1,0]
        else: # label_temp == 2
            label = [0,0,1]
    
        image_batch_labels.append(label)

    # Convert image_batch to numpy array
    image_batch = np.array(image_batch) 
    # Conver image_batch_labels to numpy array
    image_batch_labels = np.array(image_batch_labels) 
    
    return image_batch, image_batch_labels



def generate_cutmix_image(image_batch, image_batch_labels, beta):
    """ Generate a CutMix augmented image from a batch 
    Args:
        - image_batch: a batch of input images
        - image_batch_labels: labels corresponding to the image batch
        - beta: a parameter of Beta distribution.
    Returns:
        - CutMix image batch, updated labels
    """
    # generate mixed sample
    lam = np.random.beta(beta, beta)
    rand_index = np.random.permutation(len(image_batch))
    target_a = image_batch_labels
    target_b = image_batch_labels[rand_index]
    bbx1, bby1, bbx2, bby2 = rand_bbox(image_batch[0].shape, lam)
    image_batch_updated = image_batch.copy()
    image_batch_updated[:, bbx1:bbx2, bby1:bby2, :] = image_batch[rand_index, bbx1:bbx2, bby1:bby2, :]
    
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (image_batch.shape[1] * image_batch.shape[2]))
    label = target_a * lam + target_b * (1. - lam)
    
    return image_batch_updated, label



image_batch, image_batch_labels = make_img_batch_labels(image_list)
image_batch_updated, image_batch_labels_updated = generate_cutmix_image(image_batch, image_batch_labels, 0.6)

def cutmix_image(image_list):
    for i,path in tqdm(enumerate(image_list)):
        image = image_batch_updated[i]
        img_name = path.split('/')[-1].split('.')[0]
        cv2.imwrite(f"/content/aug_image/{img_name}_cutmix.png", image)

# cutmix_image(image_list)


